In [ ]:
# kết nối ggdrive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=f5590c74cd26f3ff8ec2897b554fce10dc0addad7d0fe1264b1e59aa9eaa3bc1
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [ ]:
pip install pyspark

In [ ]:
# Import các thư viện cần thiết
from pyspark import SparkContext, SparkConf
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, udf
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [ ]:
MAIN_PATH = "/content/gdrive/MyDrive/BigData_FlightDelay"
DATA_TRAIN_PATH = os.path.join(MAIN_PATH,"train_data.csv")
DATA_TEST_PATH = os.path.join(MAIN_PATH,"test_data.csv")

In [ ]:
spark = SparkSession.builder.appName("Spark Solfmax").config("spark.some.config.option", "some-value").getOrCreate()

In [ ]:
data = spark.read.option("header", True).csv(DATA_TRAIN_PATH)

In [ ]:
data.show(20)

+--------------------+-----+
|            features|LABEL|
+--------------------+-----+
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    1|
|(702,[0,1,2,3,4,5...|    1|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    2|
|(702,[0,1,2,3,4,5...|    1|
|(702,[0,1,2,3,4,5...|    1|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    2|
|(702,[0,1,2,3,4,5...|    2|
|(702,[0,1,2,3,4,5...|    2|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    0|
|(702,[0,1,2,3,4,5...|    2|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# Function to parse the vector string dynamically
def parse_vector_string(vector_str):
    vector_str = vector_str.strip('()')
    # Split into size, indices, and values
    parts = vector_str.split(',[', 2)
    size = int(parts[0].strip('['))
    indices = list(map(int, parts[1].strip(']').split(',')))
    values = list(map(float, parts[2].strip(']').split(',')))
    full_vector = [0.0] * size
    for idx, val in zip(indices, values):
        full_vector[idx] = val
    return full_vector

# Register UDF
parse_vector_udf = udf(parse_vector_string, ArrayType(DoubleType()))



In [ ]:
def parse_vector_string_rdd(vector_str):
    vector_str = vector_str.strip('"()')
    # Split into size, indices, and values
    parts = vector_str.split(',[', 2)
    size = int(parts[0].strip('['))
    indices = list(map(int, parts[1].strip(']').split(',')))
    values = list(map(float, parts[2].strip(']').split(',')))
    full_vector = [0.0] * size
    for idx, val in zip(indices, values):
        full_vector[idx] = val
    return full_vector

In [ ]:
fileRDD = spark.sparkContext.textFile(os.path.join(DATA_TRAIN_PATH))
# Skip the header and split the lines into lists of strings
dataRDD = fileRDD.filter(lambda line: 'features,LABEL' not in line).map(lambda line: (parse_vector_string_rdd(line.split('",')[0]),int(line.split('",')[1])))

In [ ]:
testRDD = spark.sparkContext.textFile(os.path.join(DATA_TEST_PATH))
# Skip the header and split the lines into lists of strings
datatestRDD = testRDD.filter(lambda line: 'features,LABEL' not in line).map(lambda line: (parse_vector_string_rdd(line.split('",')[0]),int(line.split('",')[1])))

In [ ]:
# Add parsed feature column to DataFrame
data_train = data.withColumn("features", parse_vector_udf(data.features))
data_train=data_train.withColumn("LABEL", col("LABEL").cast("int"))
#show
data_train.show()

+--------------------+-----+
|            features|LABEL|
+--------------------+-----+
|[1.0, 11.0, 1413....|    0|
|[1.0, 14.0, 608.0...|    0|
|[1.0, 16.0, 1031....|    0|
|[1.0, 30.0, 2132....|    1|
|[2.0, 11.0, 628.0...|    1|
|[2.0, 18.0, 1028....|    0|
|[2.0, 18.0, 2107....|    0|
|[3.0, 11.0, 1342....|    0|
|[3.0, 11.0, 1700....|    2|
|[3.0, 11.0, 1918....|    1|
|[3.0, 12.0, 640.0...|    1|
|[3.0, 13.0, 842.0...|    0|
|[4.0, 21.0, 1820....|    0|
|[5.0, 13.0, 936.0...|    0|
|[5.0, 14.0, 1951....|    2|
|[5.0, 24.0, 1430....|    2|
|[6.0, 12.0, 2002....|    2|
|[6.0, 15.0, 645.0...|    0|
|[6.0, 16.0, 2118....|    0|
|[6.0, 21.0, 1429....|    2|
+--------------------+-----+
only showing top 20 rows



# **Softmax Regression**

In [ ]:
def softmax(z):
    exp_z = np.exp(z - np.max(z))
    return exp_z / exp_z.sum(axis=0)

In [ ]:
def execute_softmax_regression(rdd, Weights_init, eta, tolerance=1e-4, max_count=100):
    spark_ctx = rdd.context
    Weights = Weights_init
    C = Weights_init.shape[1] #Số lớp (num_classes)
    N = rdd.count() #Số mẫu dữ liệu trong tập huấn luyện (RDD)
    input_size = len(rdd.first()[0]) #Số đặc trưng (num_features)

    count = 0
    check_weights_after = 20
    Weights_broadcast = spark_ctx.broadcast(Weights) #Chia sẻ W đến các worker node mà k cần truyền nhiều lần

    def compute_gradient(record):
        xi, label = record
        xi = np.array(xi).reshape(input_size, 1)
        yi = np.zeros((C, 1)) #vector cột y kích thước (C,1) slg lớp
        yi[label] = 1
        ai = softmax(np.dot(Weights_broadcast.value.T, xi))
        gradient = xi.dot((yi - ai).T)
        return gradient

    while count < max_count:
        gradients = rdd.map(compute_gradient).reduce(lambda g1, g2: g1 + g2)
        Weights_new = Weights + eta * gradients

        count += 1

        if count % (check_weights_after * N) == 0:
            if np.linalg.norm(Weights_new - Weights) < tolerance: #Kiểm tra điều kiện hội tụ
                return Weights_new

        Weights = Weights_new
        Weights_broadcast = spark_ctx.broadcast(Weights)

    return Weights

In [ ]:
# Khởi tạo tham số
num_features = len(dataRDD.first()[0])
num_classes = dataRDD.map(lambda x: x[1]).distinct().count() #lấy nhãn từ dataRDD khác nhau
Weights_init = np.random.randn(num_features, num_classes)
eta = 0.01
tolerance = 1e-4
max_count = 1000

In [ ]:
Weights_final = execute_softmax_regression(dataRDD, Weights_init, eta, tolerance, max_count)

In [ ]:
print("Final weights:", Weights_final)

Final weights: [[ 6.26321143e+03 -4.22380276e+02 -5.84395837e+03]
 [-5.56118676e+04  1.18685146e+04  4.37450048e+04]
 [-2.14080551e+04 -2.75053271e+04  4.89149102e+04]
 ...
 [ 5.43640690e-03  2.58197189e-03  3.88709027e-01]
 [-6.39608746e-01  1.51941750e+00  4.77937039e-01]
 [ 5.97548577e+04 -2.92422393e+04 -3.05100524e+04]]


In [ ]:
# Hàm predict label
def predict(x, Weights):
    scores = np.dot(Weights.T, x) # tính W^tX
    return np.argmax(softmax(scores), axis=0) #Dự đoán nhãn bằng cách chọn lớp có xác suất cao nhất

In [ ]:
def calculate_metrics(confusion_matrix):
    num_classes = confusion_matrix.shape[0]

    precision = np.zeros(num_classes)
    recall = np.zeros(num_classes)
    f1 = np.zeros(num_classes)

    for i in range(num_classes):
        tp = confusion_matrix[i, i]
        fp = np.sum(confusion_matrix[:, i]) - tp
        fn = np.sum(confusion_matrix[i, :]) - tp
        tn = np.sum(confusion_matrix) - (tp + fp + fn)

        precision[i] = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall[i] = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1[i] = 2 * precision[i] * recall[i] / (precision[i] + recall[i]) if (precision[i] + recall[i]) > 0 else 0

    accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix)
    macro_precision = np.mean(precision)
    macro_recall = np.mean(recall)
    macro_f1 = np.mean(f1)

    return accuracy, macro_precision, macro_recall, macro_f1

def evaluate_model(test_rdd, Weights):
    spark_ctx = test_rdd.context

    def predict_label(record):
        xi, label = record
        xi = np.array(xi).reshape(len(xi), 1)
        predicted_label = predict(xi, Weights)
        return (predicted_label, label) #trả về cặp dự đoán và nhãn thật.

    predictions_and_labels = test_rdd.map(predict_label).collect()

    predictions = [pred for pred, label in predictions_and_labels]
    true_labels = [label for pred, label in predictions_and_labels]

    num_classes = len(set(true_labels))
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

    for true, pred in zip(true_labels, predictions):
        confusion_matrix[true, pred] += 1

    accuracy, precision, recall, f1 = calculate_metrics(confusion_matrix)

    return accuracy, precision, recall, f1


In [ ]:
accuracy, precision, recall, f1 = evaluate_model(datatestRDD, Weights_final)

In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Accuracy: 0.5394996708360764
Precision: 0.28519371845025243
Recall: 0.3384095738905177
F1-score: 0.2597318531835001


In [ ]:
spark.stop()